In [1]:
# chatBot
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

In [2]:
%pip install lxml 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [2]:

raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Tennis')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [3]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

In [4]:
print(article_text)

 tennis is a racket sport that is played either individually against a single opponent (singles) or between two teams of two players each (doubles). each player uses a tennis racket strung with a cord to strike a hollow rubber ball covered with felt over or around a net and into the opponent's court. the object is to manoeuvre the ball in such a way that the opponent is not able to play a valid return. if a player is unable to return the ball successfully, the opponent scores a point. playable at all levels of society and at all ages, tennis can be played by anyone who can hold a racket, including wheelchair users. the original forms of tennis developed in france during the late middle ages. the modern form of tennis originated in birmingham, england, in the late 19th century as lawn tennis. it had close connections to various field (lawn) games such as croquet and bowls as well as to the older racket sport today called real tennis. the rules of modern tennis have changed little since 

In [7]:
import re

def custom_sent_tokenize(text):
    return re.split(r'(?<=[.!?])\s+', text)

def custom_word_tokenize(text):
    return re.findall(r'\b\w+\b', text)

article_sentences = custom_sent_tokenize(article_text)
article_words = custom_word_tokenize(article_text)

print(article_sentences)
print(article_words)


[' tennis is a racket sport that is played either individually against a single opponent (singles) or between two teams of two players each (doubles).', "each player uses a tennis racket strung with a cord to strike a hollow rubber ball covered with felt over or around a net and into the opponent's court.", 'the object is to manoeuvre the ball in such a way that the opponent is not able to play a valid return.', 'if a player is unable to return the ball successfully, the opponent scores a point.', 'playable at all levels of society and at all ages, tennis can be played by anyone who can hold a racket, including wheelchair users.', 'the original forms of tennis developed in france during the late middle ages.', 'the modern form of tennis originated in birmingham, england, in the late 19th century as lawn tennis.', 'it had close connections to various field (lawn) games such as croquet and bowls as well as to the older racket sport today called real tennis.', 'the rules of modern tennis 

In [8]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [9]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [10]:
# %pip install --force-reinstall scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
def generate_response(user_input):
    tennisrobo_response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(article_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        tennisrobo_response = tennisrobo_response + "I am sorry, I could not understand you"
        return tennisrobo_response
    else:
        tennisrobo_response = tennisrobo_response + article_sentences[similar_sentence_number]
        return tennisrobo_response

In [13]:
import re

def get_processed_text(text):
    return re.findall(r'\b\w+\b', text.lower())  # Returns list of words

word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(article_sentences)


In [14]:
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

In [15]:
similar_sentence_number = similar_vector_values.argsort()[0][-2]

In [16]:
continue_dialogue = True
print("Hello, I am your friend TennisRobo. You can ask me any question regarding tennis:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("TennisRobo: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("TennisRobo: " + generate_greeting_response(human_text))
            else:
                print("TennisRobo: ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("TennisRobo: Good bye and take care of yourself...")

Hello, I am your friend TennisRobo. You can ask me any question regarding tennis:
TennisRobo: I am sorry, I could not understand you
TennisRobo: there are also tournaments for players with disabilities, such as wheelchair tennis and deaf tennis.
TennisRobo: I am sorry, I could not understand you
TennisRobo: there were different rules at each club.
TennisRobo: one player is designated the server, and the opposing player is the receiver.
TennisRobo: Most welcome
